# Lab 4.2: Seasonal ARIMA

#### This lab is semi-open ended, in that there are many different strategies that can be used when tuning the model. As long as you thoughtfully evaluate your models and document your processes you're on the right track. Every transformation and tuning should have a visualization or other EDA process to inform the actions taken.

- Load in the data and perform EDA. 
- Record your observations regarding trend, seasonality, stationarity, etc.
- Clean the data if needed and perform any transformations you deem necessary. Don't forget to document the decisions you made, and why you made them!
- Be sure to plot ACF/PACF along the way, and record any observations you may have.
- Keep a log of each model you audition, along with their AIC values. You should be auditioning at least half a dozen models.
- Determine if Seasonal Differencing is needed. Make a note as to your decision, with graphical evidence to support.
- Clearly address the logic behind your choices for all 7 terms (3 seasonal, 3 non-seasonal, and seasonal periods).
- Utilize the Ljung-Box test to determine fit for your model. Record all observations.
- Provide both in-sample and out-of-sample forecasts for at least 3 different models. Summarize the resulting RSME values in your writeup.

### Load in the data

In [2]:
import pandas as pd
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

df = pd.read_csv('../../assets/data/h02.csv', header=None, names=['Date', 'Sales'])
df.set_index('Date', inplace=True)
df.index = df.index.to_datetime()

### Transformations
When graphed, a small increase in the variance is observed, indicating the need for logarithmic transformation.
There is a strong seasonal component, so we'd want to use seasonal differencing as well.

In [3]:
df_log = df.apply(np.log)
df_log_diff = df_log.diff(periods=12)[12:]

In [4]:
plot_acf(df_log_diff, lags=30)
plot_pacf(df_log_diff, lags=30)

ACF: No observable phenomenon at seasonal lags
PACF: Spikes at seasonal lags 12, 24. 3 spikes in non-seasonal lags. 

Interpretation: Seasonal AR(2), Non-Seasonal AR(3). ACF gives little information.

### Initial Model Audition and Tuning

In [6]:
import statsmodels.api as sm

# NOTE: THIS IS JUST ONE MODEL YOU CAN BEGIN WITH.

model = sm.tsa.statespace.SARIMAX(df_log, order=(3,0,0), seasonal_order=(2,1,0,12))
results = model.fit()
results.summary() # -> AIC:  -475.577, LB(Q) -> 59.03

# NOTE: TUNE THE MODEL ONE TERM AT A TIME, MAKING TINY CHANGES. 
# IF YOU HAVE A DIFFERENT END RESULT, IT DOESN'T MEAN YOU ARE WRONG, AS LONG 
# AS YOU MAKE IMPROVEMENTS OVER YOUR INITIAL MODEL.

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                              Sales   No. Observations:                  204
Model:             SARIMAX(3, 0, 0)x(2, 1, 0, 12)   Log Likelihood                 243.789
Date:                            Fri, 03 Jun 2016   AIC                           -475.577
Time:                                    00:55:08   BIC                           -455.669
Sample:                                07-03-1991   HQIC                          -467.524
                                     - 06-03-2008                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0981      0.066      1.494      0.135      -0.031       0.227
ar.L2          0.4063      0.049      8.371      0.000       0.311       0.501
ar.L3          0.4308      0.063      6.847      0.000       0.307       0.554
ar.S.L12      -0.4366      0.075     -5.830      0.000      -0.583      -0.290
ar.S.L24      -0.2897      0.087     -3.334      0.001      -0.460      -0.119
sigma2         0.0045      0.000     12.050      0.000       0.004       0.005
===================================================================================
Ljung-Box (Q):                       59.04   Jarque-Bera (JB):                13.26
Prob(Q):                              0.03   Prob(JB):                         0.00
Heteroskedasticity (H):               1.55   Skew:                             0.06
Prob(H) (two-sided):                  0.08   Kurtosis:                         4.28
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Final Model

In [7]:
# This is the model that I found to have the best AIC and Ljung-Box Q value.

model = sm.tsa.statespace.SARIMAX(df_log, order=(3,0,1), seasonal_order=(0,1,2,12))
results = model.fit()
results.summary() # -> AIC: -486.079, LB(Q) -> 50.95

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                              Sales   No. Observations:                  204
Model:             SARIMAX(3, 0, 1)x(0, 1, 2, 12)   Log Likelihood                 250.042
Date:                            Fri, 03 Jun 2016   AIC                           -486.085
Time:                                    00:55:14   BIC                           -462.858
Sample:                                07-03-1991   HQIC                          -476.689
                                     - 06-03-2008                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.1614      0.160     -1.009      0.313      -0.475       0.152
ar.L2          0.5487      0.080      6.860      0.000       0.392       0.705
ar.L3          0.5680      0.094      6.026      0.000       0.383       0.753
ma.L1          0.3839      0.181      2.121      0.034       0.029       0.739
ma.S.L12      -0.5223      0.074     -7.030      0.000      -0.668      -0.377
ma.S.L24      -0.1768      0.092     -1.912      0.056      -0.358       0.004
sigma2         0.0041      0.000     11.123      0.000       0.003       0.005
===================================================================================
Ljung-Box (Q):                       50.81   Jarque-Bera (JB):                 5.32
Prob(Q):                              0.12   Prob(JB):                         0.07
Heteroskedasticity (H):               1.58   Skew:                             0.03
Prob(H) (two-sided):                  0.07   Kurtosis:                         3.81
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""